# Main Telescope EFD Analysis

This notebook is based off of `../MTMount_EFDAnalysis.ipynb` and more detail is in there

This notebook seeks make histograms of MTMount velocity, acceleration, and jerk for long date ranges

This notebook extracts data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

This analysis will verify requirement 2.2.2 Slewing Rates in LTS-103 by comparing to accelerometer and encoder data. Plot distributiosn of each and superimpose the requirement spec. Need to explain/analyse ALL outliers as this is a glass safety issue. 

Go back into the historical data - check since begining of time.  ANALYSE  a night, and all data 
If no anomalies are found, we can set an upper limit and it must be monitored going forward. Passing this allows for acceptance of TMA from vendor but this must be continually monitored. 

If anomaliy - look for faults/messages from the MTMount CSC (Russel Owen). These can be found in the EFD - look at topics - like the encoder values 

ts-xml.lsst.io - describes the schema for the EFD and the values to extract. 

Ask Holger which test case this is associated with. 

This addresses 
5.  check slewing rates (velocity, acceleration, jerk) and motion profiles from all available previous historical telemetry. 
-- also need to cross check with DIMM data - mainly for jitter if we see accelleration problems. Can use DIMM to check and do a follow on data. DIMM data is in the LFA. This is a follow on post TMA acceptance. 

Main goal: Need to see accelerometer data coincide with encoder data.

Ask qusestions on #rubinobs-sitcom-startracker

2,3,6 have no effort 
2 

In [ ]:
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio
import numpy as np
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample
from scipy.interpolate import interp1d
import math

In [ ]:
# Change client if using USDF. 
client = EfdClient('usdf_efd')
client.output = 'dataframe'

We'll access the DM-EFD instance deployed at the summit. You need to be on site or connected to the NOAO VPN. 

## Declare timestamps used for EFD queries

In [ ]:
### Example date range
t1 = Time('2023-02-01T02:35:01', scale='utc') 
# window = TimeDelta(90*24*60*60, format='sec')
# t2=t1+window
### Example 
#t1 = Time('2022-12-00T02:35:01', scale='utc') 
#window = TimeDelta(4, format='sec')
#t2=t1+window
t2 = Time('2023-03-17T02:35:01', scale='utc') 


In [ ]:
### Date range for the formal verification 

In [ ]:
# time check verification
print(t1.isot)
print(t1.datetime64)
print(t2.isot)

# Load EFD data
For this I have grabbed just the actualPosition and actualVelocity values for the alt and elevation encoders

For a larger query (longer time range) may want to run query as a script and load results.

See link for example query script in March 8th on #rubinobs-mtmount: https://lsstc.slack.com/archives/C047LHXRB4K/p1678251239301319

In [ ]:
# Measured positions and velocities 
MTMount_Az = await client.select_time_series("lsst.sal.MTMount.azimuth", 
                                                  ["actualPosition","actualVelocity"], 
                                                  t1, t2)
MTMount_El = await client.select_time_series("lsst.sal.MTMount.elevation", ["actualPosition","actualVelocity"] ,t1, t2)
            

## LTS-103 2.2.2 Slewing rate 

This section responds to requirements in LTS-103, 2.2.2 Slewing Rates

2.2.2.1 Maximum Slewing Rate 

Specification: The slew and settling time requirements shall be met without exceeding the maximum
limits, if these values are exceeded the telescope mount assembly shall be automatically stopped per
the requirements of section 3.7.7:

### Define relevant specifications

In [ ]:
# Azimuth velocity min/max: ±10.5 deg/sec
SPEC_AZ_VEL_MAX = 10.5

# Azimuth acceleration: ±10.5 deg/s2
SPEC_AZ_ACC_MAX = 10.5

# Azimuth jerk: ±42.0 deg/s3
SPEC_AZ_JERK_MAX = 42.0 

# Elevation velocity: ±5.25 deg/s
SPEC_ELEV_VEL_MAX = 5.25

# Elevation acceleration: ±5.25 deg/s2
SPEC_ELEV_ACC_MAX = 5.25

# Elevation jerk: ±21.0 deg/s3
SPEC_ELEV_JERK_MAX  = 21.0 

# Calculation of Acceleration/Jerk
Combine into single data frame

In [ ]:
calculated_frame_el = pd.DataFrame({"time_el":MTMount_El.index,
                              "relative_time_el":(MTMount_El.index - MTMount_El.index[0]).total_seconds(),
                              "position_el":MTMount_El["actualPosition"].values,
                              "velocity_el":MTMount_El["actualVelocity"].values,
                              })

In [ ]:
# Input is a matrix with n lines and 2 columns
# def addInQuadrature(points):
#     centroid = np.mean(points, axis=0)
#     return np.sum(np.hypot(points[:,0]-centroid[0], points[:,1]-centroid[1]))

# testPoints = np.
# assert addInQuadrature(testPoints)

In [ ]:
az_pos_interp=interp1d(calculated_frame_el["relative_time_el"], calculated_frame_el["position_el"], bounds_error=False)


calculated_frame_az = pd.DataFrame({"time_az":MTMount_Az.index,
                              "relative_time_az":(MTMount_Az.index - MTMount_Az.index[0]).total_seconds(),
                              "position_az":MTMount_Az["actualPosition"].values,
                              
                              })
calculated_frame_az["velocity_az"]=MTMount_Az["actualVelocity"].values * np.cos(np.radians(az_pos_interp(calculated_frame_az["relative_time_az"])))

calculated_frame_az["acceleration_az"] = np.gradient(calculated_frame_az["velocity_az"],
                                                    calculated_frame_az["relative_time_az"]) 
calculated_frame_az["jerk_az"] = np.gradient(calculated_frame_az["acceleration_az"],
                                            calculated_frame_az["relative_time_az"]) 


calculated_frame_el["acceleration_el"] = np.gradient(calculated_frame_el["velocity_el"],
                                                    calculated_frame_el["relative_time_el"]) 
calculated_frame_el["jerk_el"] = np.gradient(calculated_frame_el["acceleration_el"],
                                            calculated_frame_el["relative_time_el"]) 


calculated_frame_total = pd.DataFrame({"time_total":MTMount_Az.index,
                              "relative_time_total":(MTMount_Az.index - MTMount_Az.index[0]).total_seconds(),
                              
                              })
el_velocity_interp=interp1d(calculated_frame_el["relative_time_el"], calculated_frame_el["velocity_el"], bounds_error=False)
calculated_frame_total["theta"]=np.arctan(MTMount_Az["actualVelocity"].values/el_velocity_interp(calculated_frame_total["relative_time_total"].values))
calculated_frame_total["velocity_total"]=np.sqrt(calculated_frame_az["velocity_az"].values**2 +
                                                 el_velocity_interp(calculated_frame_total["relative_time_total"].values)**2)
calculated_frame_total["acceleration_total"]=np.gradient(calculated_frame_total["velocity_total"],
                                            calculated_frame_total["relative_time_total"]) 
calculated_frame_total["jerk_total"] = np.gradient(calculated_frame_total["acceleration_total"],
                                            calculated_frame_total["relative_time_total"]) 

calculated_frame=pd.concat([calculated_frame_az,calculated_frame_el, calculated_frame_total], axis=1)

In [ ]:
calculated_frame.head()

## Movement Histograms
See sitcom 710: https://jira.lsstcorp.org/browse/SITCOM-710 

- I think we just want full histograms for Alt, El, and (maybe total)
- I also *think* for total speed (velocity) we need a cos(elevation) factor, but havent fully thought that through
- I dont think position makes much sense here
- I also dont know that the limits are but was planning on marking those for each hist with vertical lines
- could remove the spike close to zero with a different sel_slew (2 cells below)

In [ ]:
# setup for plotting 
mtypes=["az","el","total"]
color_dict={"total":"k",
            "az":"tab:red",
            "el":"tab:blue"}
label_dict={"total":"Total (speed)",
            "az": "Azmiuth",
            "el": "Elevation"}

In [ ]:
# selecting only times where the telescope is slewing
sel_slew=(abs(calculated_frame["velocity_az"]) > 0.04) | (abs(calculated_frame["velocity_el"]) > 0.04)

In [ ]:
# create sensible bin sizes to cover full range
vel_list=[]
acc_list=[]
jerk_list=[]
for i in mtypes:
    vel_list.append(calculated_frame[f"velocity_{i}"])
    acc_list.append(calculated_frame[f"acceleration_{i}"])
    jerk_list.append(calculated_frame[f"jerk_{i}"])
    


vel_bins=np.arange(np.nanmin(vel_list),np.nanmax(vel_list),0.5)
acc_bins=np.arange(np.nanmin(acc_list),np.nanmax(acc_list),0.1)
jerk_bins=np.arange(np.nanmin(jerk_list),np.nanmax(jerk_list),1)


In [ ]:
yMax=0

fig,axs=plt.subplots(1,3,dpi=175, figsize=(13,5))
plt.suptitle(f"TMA movements\n {t1.iso[:10]} : {t2.iso[:10]}")
for i in mtypes:
    velHist=axs[0].hist(calculated_frame[f"velocity_{i}"][sel_slew], 
                bins=vel_bins, histtype="step", color=color_dict[f"{i}"])
    yMax=np.max([yMax,np.max(velHist[0])])


axs[0].set_xlabel("Velocity [deg/s]")

for i in mtypes:
    accHist=axs[1].hist(calculated_frame[f"acceleration_{i}"][sel_slew], 
                bins=acc_bins, histtype="step", color=color_dict[i])
    yMax=np.max([yMax,np.max(accHist[0])])
        
axs[1].set_xlabel(r"Acceleration [deg/s$^2$]")

for i in mtypes:
    jerkHist=axs[2].hist(calculated_frame[f"jerk_{i}"][sel_slew], 
                bins=jerk_bins, histtype="step", color=color_dict[i], label=label_dict[i])
    yMax=np.max([yMax,np.max(jerkHist[0])])
        
axs[2].set_xlabel(r"Jerk [deg/s$^3$]")
axs[2].legend()

#yMax=10e5
for i in [0,1,2]:
    ax=axs[i]
    ax.set_yscale('log')
    ax.set_ylim(1e0,yMax * 1.2)
    if i > 0:
        ax.set_yticklabels([])
    else:
        ax.set_ylabel("count")
plt.subplots_adjust(wspace=0)